### DNN 기반 이진분류 모델 구현 <hr>
- 데이터셋 : iris.csv
- 피쳐/속성 : 4개 Sepal_Length, Sepan_Width, Petal_ Length, Peatal_Width
- 타겟/라벨 : 1개 Setosa와 나머지
- 학습방벙 : 지도학습 > 분류 > 이진 분류
- 알고리즘 : 인공신경망 (ANN) -> MLP(Multi Layer Percetron),  심층신경망(DNN)(은닉층 多)
- 프레임워크 : Pytorch
---

[1] 모듈 로딩 및 데이터 준비  <hr>

In [78]:
# 모듈로딩
# Model 관련
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
from torchmetrics.classification import  F1Score, BinaryF1Score, BinaryConfusionMatrix
from torchinfo import summary

# - Data관련
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import *
from sklearn.model_selection import train_test_split

In [79]:
# 활용 패키지 버전 체크 - 사용자 정의 함수로 구현할것!
print(f'Pytorch v.{torch.__version__}')
print(f'Pandas v.{pd.__version__}')

Pytorch v.2.4.1
Pandas v.2.0.3


In [80]:
## 데이터로딩
DATA_FILE='../../MachineLearning/data/iris.csv'

## CSV >> DataFrame
irisDF=pd.read_csv(DATA_FILE)

## 확인
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [81]:
### 타겟 변경 => 정수화, 클래스3개 =>2개
irisDF['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [82]:
irisDF['variety']=(irisDF['variety']=='Setosa')
irisDF.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,True
1,4.9,3.0,1.4,0.2,True
2,4.7,3.2,1.3,0.2,True
3,4.6,3.1,1.5,0.2,True
4,5.0,3.6,1.4,0.2,True


In [83]:
irisDF['variety']=irisDF['variety'].astype('int')
irisDF['variety'].unique()

array([1, 0])

In [84]:
# 타겟 정수화
labels=dict(zip(irisDF['variety'].unique().tolist(),range(3)))
print(f'labels=> {labels}')
# 라벨인코딩<<

labels=> {1: 0, 0: 1}


[2] 모델 클래스 설계 및 정의 <hr>
- 클래스목적 : iris 데이터를 학습 및 추론 목적
- 클래스이름 : IrisBCFModel
- 부모클래스 : nn.Module
- 매개변수 : 층별 입출력 갯수 고정하므로 필요 없음
- 속성/필드 : featruesDF, targetDF, n_rows, n_features
- 기능/역할 <필수 >
    -  __ init __() : 모델 구조 설정 
    - forward() : 순방향 학습 <- 오버라이딩(overriding)
- 클래스구조
    - 입력층 : 입력 4개 / 출력 10개 (퍼셉트론/뉴런 10개 존재)
    - 은닉층 : 입력 10개 / 출력 5개 (퍼셉트론/뉴런 30개 존재) 
    - 출력층 : 입력 5개 / 출력 1개 (2진분류)
- 손실함수/ 활성화함수 
    - 클래스형태 => nn. 아래  
        - ex) nn.ReLu << __init__() 에 사용
    - 함수 형태 => torch.nn.functional 아래
        - ex) F.relu << def forward() 에 사용


In [85]:
class IrisBCFModel(nn.Module):

    # 모델 구조 구성 및 인스턴스 생성 메서드
    def __init__(self):
        super().__init__()

        self.in_layer=nn.Linear(4,10)
        self.hd_layer=nn.Linear(10,5)
        self.out_layer=nn.Linear(5,1)

    # 순방향 학습 진행 메서드
    def forward(self,x):
        # - 입력층
        y=self.in_layer(x)        # f1_1w1_1+f1_2w1_2+f1_3w1_3+b << 이런식 10개
        y=F.relu(y)

        # - 은닉층 : 10개의 숫자값 (>=0)
        y=self.hd_layer(y)        # f2_1w2_1+f2_2w2_2+....f2_10w2_10 +b << 이런식 5개
        y=F.relu(y)

        # - 출력층 : 5개의 숫자값 (>=0) - 이진분류
        return F.sigmoid(self.out_layer(y))         # f3_1w3_1+.....f3_30w3_30+b << 1 개



In [86]:
### [테스트] 모델 인스턴스 생성
model=IrisBCFModel()

print(model)

IrisBCFModel(
  (in_layer): Linear(in_features=4, out_features=10, bias=True)
  (hd_layer): Linear(in_features=10, out_features=5, bias=True)
  (out_layer): Linear(in_features=5, out_features=1, bias=True)
)


In [87]:
### [테스트] 모델 확인
summary(model,input_size=(17,4))

Layer (type:depth-idx)                   Output Shape              Param #
IrisBCFModel                             [17, 1]                   --
├─Linear: 1-1                            [17, 10]                  50
├─Linear: 1-2                            [17, 5]                   55
├─Linear: 1-3                            [17, 1]                   6
Total params: 111
Trainable params: 111
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

[3] 데이터셋 클래스 설계 및 정의 <hr>
- 데이터셋 : iris.csv
- 피쳐개수 : 4개
- 타겟개수 : 1개
- 클래스이름 : IrisDataset
- 부모클래스 : utils.data.Dataset
- 속성필드 : featureDF, targetDF, n_rows, n_features
- 필수메서드 
    - __ init __(self) : 데이터셋 저장 및 전처리, 개발자가 필요한 속성 설정
    - __ len __(self) : 데이터의 개수 반환
    - __ getitem __(self,index) : 특정 인덱스의 피쳐와 타겟 반환

In [88]:
class IrisDataset(Dataset):

    def __init__(self,featureDF,targetDF):
        self.featureDF=featureDF
        self.targetDF=targetDF
        self.n_rows=featureDF.shape[0]      # 필수아님
        self.n_features=featureDF.shape[1]  # 필수아님


    def __len__(self):
        return self.n_rows
        
    def __getitem__(self, index):
        # 텐서화
        featureTS=torch.FloatTensor(self.featureDF.iloc[index].values)
        targetTS=torch.FloatTensor(self.targetDF.iloc[index].values)
        
        # 피쳐와 타겟 반환
        return featureTS, targetTS
    

In [89]:
## [테스트] 데이터셋 인스턴스 생성

featureDF=irisDF[irisDF.columns[:4]]        # 2D (150,3)
targetDF=irisDF[irisDF.columns[[4]]]        # 2D (150,1)

# - 커스템데이터셋 인스턴스 생성
irisDS=IrisDataset(featureDF,targetDF)

# 데이터 로드 인스턴스 생성
irisDL=DataLoader(irisDS)
for feature,target in irisDL:
    print(feature,target)
    break


tensor([[5.1000, 3.5000, 1.4000, 0.2000]]) tensor([[1.]])


[4] 학습 준비
- 학습횟수 : EPOCH <- 처음부터 끝까지 공부하는 횟수
- 배치크기 : BATCH_SIZE <- 한번에 학습할 데이터셋 양
- 위치지정 : DEVICE <- 텐서 저장 및 실행 위치 (GPU/CPU)
- 학습률(LR, learning Rate) : 가중치와 절편 업데이트 시 경사하강법으로 업데이트 간격 설정 0.001~0.1

In [100]:
### 학습 진행 관련 설정 값 
EPOCH = 100
BATCH_SIZE= 10
DEVICE ='cuda' if torch.cuda.is_available() else 'cpu'
LR=0.001

- 인스턴스/객체 : 모델, 데이터셋, 최적화, 손실함수 (, 성능지표)

In [91]:
# 모델 인스턴스
model= IrisBCFModel()

# 데이터셋 인스턴스
# # 학습,검증,테스트 데이터 분리
X_train, X_test, y_train, y_test=train_test_split(featureDF,targetDF,random_state=1)
X_train,X_val,y_train,y_val=train_test_split(X_train,y_train,random_state=1)
print(f' X_train : {X_train.shape}, X_val : {X_val.shape} , X_test : {X_test.shape}')
print(f' y_train : {y_train.shape}, y_val : {y_val.shape} , y_test : {y_test.shape}')



trainDS=IrisDataset(X_train,y_train)
valDS=IrisDataset(X_val,y_val)
testDS=IrisDataset(X_test,y_test)

# 데이터로더 인스턴스
trainDL=DataLoader(trainDS,batch_size=BATCH_SIZE)

 X_train : (84, 4), X_val : (28, 4) , X_test : (38, 4)
 y_train : (84, 1), y_val : (28, 1) , y_test : (38, 1)


- 최적화 , 손실함수 인스턴스 생성

In [97]:
# 최적화 인스턴스 => W,b 텐서 즉, model.parameters() 전달
optimizer=optim.Adam(model.parameters(),lr=LR)

# 손실함수 인스턴스 => 분류 => 이진분류 BinaryCrossEntropyLoss => BCELoss
#                            예측값은 확률값으로 전달 => sigmoid() AF처리후 전달
bceLoss=nn.BCELoss()

[5] 학습진행

In [98]:
len(trainDL),trainDL.__len__()

(9, 9)

In [102]:
## 학습의 효과 확인 손실값과 성능 평가값 저장 필요
LOSS_HISTORY,SCORE_HISTORY=[[],[]],[[],[]]

# 학습 모드로 모델 설정
 
for epoch in range(EPOCH):
    model.train()  ## 학습모드 켜기! 에포크 단위로 학습과 검증 진행시 for문안에서 껐다 켯다 해야함! 

    # 배치 크기 만큼 데이터 로딩해서 학습 진행
    loss_total=0
    score_total=0
    for featureTS,targetTS in trainDL:

        # 학습 진행
        pre_y=model(featureTS)

        # 손실 계산
        loss=bceLoss(pre_y,targetTS)
        loss_total+=loss.item()

        # 성능평가 계산
        score=BinaryF1Score()(pre_y,targetTS)
        score_total+=score.item()

        # 최적화 진행
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    model.eval() # 검증모드 켜기!
    with torch.no_grad():
        # 검증데이터셋
        val_featureTS=torch.FloatTensor(valDS.featureDF.values)
        val_targetTS=torch.FloatTensor(valDS.targetDF.values)

        # 평가
        pre_val = model(val_featureTS)
        #손실
        val_loss=bceLoss(pre_val,val_targetTS)
        # 성능평가
        val_score=BinaryF1Score()(pre_val,val_targetTS)
    
    # 손실값과 성능 평가값 저장
    LOSS_HISTORY[0].append(loss_total/len(trainDL))
    SCORE_HISTORY[0].append(score_total/len(trainDL))

    LOSS_HISTORY[1].append(val_loss)
    SCORE_HISTORY[1].append(val_score)

    print(f'[{epoch}/{EPOCH}]')
    print(f'trainloss={loss_total/len(trainDL)} trainscore={score_total/len(trainDL)}')
    print(f'valloss={val_loss} trainscore={val_score}')

[0/100]
trainloss=0.0070068777487095855 trainscore=1.0
valloss=0.00737631693482399 trainscore=1.0
[1/100]
trainloss=0.006805177079513669 trainscore=1.0
valloss=0.007156603038311005 trainscore=1.0
[2/100]
trainloss=0.006612745605202185 trainscore=1.0
valloss=0.006947422865778208 trainscore=1.0
[3/100]
trainloss=0.006428463410379158 trainscore=1.0
valloss=0.006748297717422247 trainscore=1.0
[4/100]
trainloss=0.00625084594099058 trainscore=1.0
valloss=0.006556288804858923 trainscore=1.0
[5/100]
trainloss=0.006080331824099024 trainscore=1.0
valloss=0.006372309289872646 trainscore=1.0
[6/100]
trainloss=0.005916911901699172 trainscore=1.0
valloss=0.0061953989788889885 trainscore=1.0
[7/100]
trainloss=0.0057592792177779805 trainscore=1.0
valloss=0.006025427021086216 trainscore=1.0
[8/100]
trainloss=0.005608680269991358 trainscore=1.0
valloss=0.00586332380771637 trainscore=1.0
[9/100]
trainloss=0.005462837353762653 trainscore=1.0
valloss=0.005706308875232935 trainscore=1.0
[10/100]
trainloss=0

- 학습결과 체크 => 학습과 검증의 Loss변화, 성능변화